<a href="https://colab.research.google.com/github/RithishChowdary/Stock-Predicting/blob/main/Stock_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [17]:
price_change = ctrl.Antecedent(np.arange(-10,11,1),'price_change')
volume = ctrl.Antecedent(np.arange(0,101,1),'volume')
sentiment = ctrl.Antecedent(np.arange(-1,1.1,0.1),'sentiment')
trend=ctrl.Consequent(np.arange(0,101,1),'trend')

In [18]:
price_change['low'] = fuzz.trimf(price_change.universe, [-10, -10, 0])
price_change['medium'] = fuzz.trimf(price_change.universe, [-5, 0, 5])
price_change['high'] = fuzz.trimf(price_change.universe, [0, 10, 10])

In [19]:
volume['low'] = fuzz.trimf(volume.universe, [0, 0, 30])
volume['medium'] = fuzz.trimf(volume.universe, [20, 50, 80])
volume['high'] = fuzz.trimf(volume.universe, [70, 100, 100])

In [20]:
sentiment['negative'] = fuzz.trimf(sentiment.universe, [-1, -1, 0])
sentiment['neutral'] = fuzz.trimf(sentiment.universe, [-0.5, 0, 0.5])
sentiment['positive'] = fuzz.trimf(sentiment.universe, [0, 1, 1])

In [21]:
trend['bearish'] = fuzz.trimf(trend.universe, [0, 0, 50])
trend['stable'] = fuzz.trimf(trend.universe, [25, 50, 75])
trend['bullish'] = fuzz.trimf(trend.universe, [50, 100, 100])

In [22]:
rule1 = ctrl.Rule(price_change['high'] & volume['high'] & sentiment['positive'], trend['bullish'])
rule2 = ctrl.Rule(price_change['low'] & volume['high'] & sentiment['negative'], trend['bearish'])
rule3 = ctrl.Rule(price_change['medium'] & volume['medium'] & sentiment['neutral'], trend['stable'])
rule4 = ctrl.Rule(price_change['low'] & volume['low'] & sentiment['negative'], trend['bearish'])
rule5 = ctrl.Rule(price_change['high'] & volume['low'] & sentiment['positive'], trend['bullish'])

In [24]:
trend_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
trend_predictor = ctrl.ControlSystemSimulation(trend_ctrl)

In [25]:
try:
    user_price = float(input("Enter price change (% between -10 to 10): "))
    user_volume = float(input("Enter volume (in millions, 0 to 100): "))
    user_sentiment = float(input("Enter sentiment (-1 to 1): "))

    if not (-10 <= user_price <= 10):
        raise ValueError("Price change must be between -10 and 10.")
    if not (0 <= user_volume <= 100):
        raise ValueError("Volume must be between 0 and 100.")
    if not (-1 <= user_sentiment <= 1):
        raise ValueError("Sentiment must be between -1 and 1.")

    trend_predictor.input['price_change'] = user_price
    trend_predictor.input['volume'] = user_volume
    trend_predictor.input['sentiment'] = user_sentiment


    trend_predictor.compute()
    output = trend_predictor.output['trend']

    print(f"\n Predicted Trend Score: {output:.2f}")

    if output > 66:
        print(" Final Prediction: Bullish")
    elif output < 33:
        print(" Final Prediction: Bearish")
    else:
        print(" Final Prediction: Stable")

except ValueError as ve:
    print(f" Input Error: {ve}")

Enter price change (% between -10 to 10): 1
Enter volume (in millions, 0 to 100): 100
Enter sentiment (-1 to 1): 0.5

 Predicted Trend Score: 76.23
 Final Prediction: Bullish
